In [2]:
# Mount Drive
import google.colab

from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google.colab'

In [0]:
import ee
import numpy as np
import pandas as pd
import os

service_account = 'encaramelo@alien-fold-235112.iam.gserviceaccount.com'
credentials = ee.ServiceAccountCredentials(service_account, '/content/drive/My Drive/Colab Notebooks/alien-fold-235112-e3ab4bc3ddd0.json')
ee.Initialize(credentials)

In [0]:
## Initialise system
#import ee
#import numpy as np
#import pandas as pd
#import os

## Trigger the authentication flow.
#ee.Authenticate()

## Initialise the library.
#ee.Initialize()

In [0]:
# aux functions for time
import datetime 
import time

def get_timestamp(in_date):
    utc_naive  = in_date.replace(tzinfo=None) - in_date.utcoffset()
    timestamp = (utc_naive - datetime.datetime(1970, 1, 1)).total_seconds()
    return (timestamp)


def get_timestamp_utc(in_date):
    utc_naive  = in_date.replace(tzinfo=None)
    timestamp = (utc_naive - datetime.datetime(1970, 1, 1)).total_seconds()
    return int(timestamp)

In [0]:
def fc2df(fc):
    # Convert a FeatureCollection into a pandas DataFrame
    features = fc.getInfo()['features']
    dictarr = []
    for f in features:
        attr = f['properties']
        dictarr.append(attr)

    df = pd.DataFrame(dictarr)
    return df

In [0]:
def get_image_n_box(latitude, longitude, start_time, end_time):
  """ Returns the clippled image collection and the used box """

  point = ee.Geometry.Point([longitude,latitude]);
  
  S2 = ee.ImageCollection("COPERNICUS/S2_SR");
  # Filtering
  S2TemporalFiltered = S2.filterDate(start_time,end_time);
  S2SpatialFiltered = S2TemporalFiltered.filterBounds(point);
  imagenes = S2SpatialFiltered.size()

  # Box coordinates to reduce collection
  x1 = longitude - 0.00058;
  x2 = longitude + 0.00058;
  y1 = latitude - 0.000437;
  y2 = latitude + 0.000437;
  box = ee.Geometry.Rectangle(x1,y1,x2,y2);

  # Cutting collection to box
  S2Clipped = S2SpatialFiltered.map(
  lambda img : img.clip(box)
  );

  # Mean values
  eBox = ee.Feature(box);
  return S2Clipped, eBox

In [0]:
def get_dataframes(S2Clipped, eBox):
  """ Returns Bandas and Properties dataframe """

  mbox_image = lambda image : ee.Feature(None, image.reduceRegion(reducer=ee.Reducer.mean(), geometry=eBox.geometry())) 
  get_properties = lambda image : ee.Feature(None,ee.Dictionary({
    "system_index":image.get('system:index'),
    "MEAN_SOLAR_AZIMUTH_ANGLE":image.get('MEAN_SOLAR_AZIMUTH_ANGLE'),
    "MEAN_INCIDENCE_AZIMUTH_ANGLE_B1":image.get('MEAN_INCIDENCE_AZIMUTH_ANGLE_B1'),
    "MEAN_INCIDENCE_AZIMUTH_ANGLE_B2":image.get('MEAN_INCIDENCE_AZIMUTH_ANGLE_B2'),
    "MEAN_INCIDENCE_AZIMUTH_ANGLE_B3":image.get('MEAN_INCIDENCE_AZIMUTH_ANGLE_B3'),
    "MEAN_INCIDENCE_AZIMUTH_ANGLE_B4":image.get('MEAN_INCIDENCE_AZIMUTH_ANGLE_B4'),
    "MEAN_INCIDENCE_AZIMUTH_ANGLE_B5":image.get('MEAN_INCIDENCE_AZIMUTH_ANGLE_B5'),
    "MEAN_INCIDENCE_AZIMUTH_ANGLE_B6":image.get('MEAN_INCIDENCE_AZIMUTH_ANGLE_B6'),
    "MEAN_INCIDENCE_AZIMUTH_ANGLE_B7":image.get('MEAN_INCIDENCE_AZIMUTH_ANGLE_B7'),
    "MEAN_INCIDENCE_AZIMUTH_ANGLE_B8":image.get('MEAN_INCIDENCE_AZIMUTH_ANGLE_B8'),
    "MEAN_INCIDENCE_AZIMUTH_ANGLE_B8A":image.get('MEAN_INCIDENCE_AZIMUTH_ANGLE_B8A'),
    "MEAN_INCIDENCE_AZIMUTH_ANGLE_B9":image.get('MEAN_INCIDENCE_AZIMUTH_ANGLE_B9'),
    "MEAN_INCIDENCE_AZIMUTH_ANGLE_B10":image.get('MEAN_INCIDENCE_AZIMUTH_ANGLE_B10'),
    "MEAN_INCIDENCE_AZIMUTH_ANGLE_B11":image.get('MEAN_INCIDENCE_AZIMUTH_ANGLE_B11'),
    "MEAN_INCIDENCE_AZIMUTH_ANGLE_B12":image.get('MEAN_INCIDENCE_AZIMUTH_ANGLE_B12'),
    "SOLAR_IRRADIANCE_B1":image.get('SOLAR_IRRADIANCE_B1'),
    "SOLAR_IRRADIANCE_B2":image.get('SOLAR_IRRADIANCE_B2'),
    "SOLAR_IRRADIANCE_B3":image.get('SOLAR_IRRADIANCE_B3'),
    "SOLAR_IRRADIANCE_B4":image.get('SOLAR_IRRADIANCE_B4'),
    "SOLAR_IRRADIANCE_B5":image.get('SOLAR_IRRADIANCE_B5'),
    "SOLAR_IRRADIANCE_B6":image.get('SOLAR_IRRADIANCE_B6'),
    "SOLAR_IRRADIANCE_B7":image.get('SOLAR_IRRADIANCE_B7'),
    "SOLAR_IRRADIANCE_B8":image.get('SOLAR_IRRADIANCE_B8'),
    "SOLAR_IRRADIANCE_B8A":image.get('SOLAR_IRRADIANCE_B8A'),
    "SOLAR_IRRADIANCE_B9":image.get('SOLAR_IRRADIANCE_B9'),
    "SOLAR_IRRADIANCE_B10":image.get('SOLAR_IRRADIANCE_B10'),
    "SOLAR_IRRADIANCE_B11":image.get('SOLAR_IRRADIANCE_B11'),
    "SOLAR_IRRADIANCE_B12":image.get('SOLAR_IRRADIANCE_B12'),
    "CLOUDY_PIXEL_PERCENTAGE":image.get('CLOUDY_PIXEL_PERCENTAGE'),
    "CLOUD_COVERAGE_ASSESSMENT":image.get('CLOUD_COVERAGE_ASSESSMENT'),
    "REFLECTANCE_CONVERSION_CORRECTION":image.get('REFLECTANCE_CONVERSION_CORRECTION'),
    "MEAN_SOLAR_ZENITH_ANGLE":image.get('MEAN_SOLAR_ZENITH_ANGLE '),
    "MEAN_INCIDENCE_ZENITH_ANGLE_B1":image.get('MEAN_INCIDENCE_ZENITH_ANGLE_B1'),
    "MEAN_INCIDENCE_ZENITH_ANGLE_B2":image.get('MEAN_INCIDENCE_ZENITH_ANGLE_B2'),
    "MEAN_INCIDENCE_ZENITH_ANGLE_B3":image.get('MEAN_INCIDENCE_ZENITH_ANGLE_B3'),
    "MEAN_INCIDENCE_ZENITH_ANGLE_B4":image.get('MEAN_INCIDENCE_ZENITH_ANGLE_B4'),
    "MEAN_INCIDENCE_ZENITH_ANGLE_B5":image.get('MEAN_INCIDENCE_ZENITH_ANGLE_B5'),
    "MEAN_INCIDENCE_ZENITH_ANGLE_B6":image.get('MEAN_INCIDENCE_ZENITH_ANGLE_B6'),
    "MEAN_INCIDENCE_ZENITH_ANGLE_B7":image.get('MEAN_INCIDENCE_ZENITH_ANGLE_B7'),
    "MEAN_INCIDENCE_ZENITH_ANGLE_B8":image.get('MEAN_INCIDENCE_ZENITH_ANGLE_B8'),
    "MEAN_INCIDENCE_ZENITH_ANGLE_B8A":image.get('MEAN_INCIDENCE_ZENITH_ANGLE_B8A'),
    "MEAN_INCIDENCE_ZENITH_ANGLE_B9":image.get('MEAN_INCIDENCE_ZENITH_ANGLE_B9'),
    "MEAN_INCIDENCE_ZENITH_ANGLE_B10":image.get('MEAN_INCIDENCE_ZENITH_ANGLE_B10'),
    "MEAN_INCIDENCE_ZENITH_ANGLE_B11":image.get('MEAN_INCIDENCE_ZENITH_ANGLE_B11'),
    "MEAN_INCIDENCE_ZENITH_ANGLE_B12":image.get('MEAN_INCIDENCE_ZENITH_ANGLE_B12')}))  

  df_bandas = fc2df(S2Clipped.map(mbox_image))
  df_propiedades = fc2df(S2Clipped.map(get_properties))
  return df_bandas, df_propiedades

In [0]:
def get_satellite_dataframe_row(latitude, longitude, date):
    start_time = date.strftime("%Y-%m-%d")
    end_time = (date + datetime.timedelta(days=1)).strftime("%Y-%m-%d")
    
    S2Clipped, eBox = get_image_n_box(latitude, longitude, start_time, end_time)
    df_bandas, df_propiedades = get_dataframes(S2Clipped, eBox)
    
    merged = df_bandas.join(df_propiedades)
    
    if not merged.empty:
        merged.sort_values(by=['system_index'],inplace=True)
        merged=merged.tail(1) # get last
    return merged
  

In [0]:
# get measurement dataframe (only csv for now)
def load_n_process_measurements_file(file_path):
    """ Load'n'process measurement csv and returns dataframe with rows that fall within an image. 
    Note: limited to one image per day"""
    print("Opening measurement csv file '{}'".format(file_path))
    df_measurements = pd.read_csv(file_path, header=0)
    df_measurements['DATE'] = [datetime.datetime.strptime(i[0:19], '%Y-%m-%d  %H:%M:%S').date() for i in df_measurements['DATETIME']]
    df_out = pd.DataFrame(columns=df_measurements.columns.values)
    for measurement_date in df_measurements.drop_duplicates('DATE', keep='first')['DATE']:
        df_at_measurement_date = df_measurements.loc[df_measurements['DATE'] == measurement_date]
        sat_timestamps = ee_get_picture_date(measurement_date,
                                             df_at_measurement_date["LATITUDE"].min(),
                                             df_at_measurement_date["LATITUDE"].max(),
                                             df_at_measurement_date["LONGITUDE"].min(),
                                             df_at_measurement_date["LONGITUDE"].max() )
        df_temp = pd.DataFrame()
        for sat_timestamp in sat_timestamps:
            df_dif = df_at_measurement_date
            df_dif['TIMESTAMP_DIFF'] = (df_at_measurement_date['TIMESTAMP'] - sat_timestamp).abs()
            min_dif = df_dif['TIMESTAMP_DIFF'].min()
            if min_dif < 3600:
                df_temp = pd.concat([df_temp, df_dif[df_dif['TIMESTAMP_DIFF'] == min_dif]], axis =0)
        if not df_temp.empty:        
            df_temp.sort_values(by=['TIMESTAMP_DIFF'],inplace=True)
            df_out = pd.concat([df_out, df_temp.head(1)], axis =0)
    return df_out


In [0]:
def timestamp_from_system_index(system_index):
    year = int(system_index[0:4])
    month = int(system_index[4:6])
    day = int(system_index[6:8])
    hour = int(system_index[9:11])
    minutes = int(system_index[11:13])
    return get_timestamp_utc(datetime.datetime(year, month, day, hour, minutes))


S2 = ee.ImageCollection("COPERNICUS/S2");
def ee_get_picture_date(date, min_lat, max_lat, min_lon, max_lon):
  box = ee.Geometry.Rectangle(min_lon,min_lat,max_lon,max_lat);
  start_time = date.strftime("%Y-%m-%d")
  end_time = (date + datetime.timedelta(days=1)).strftime("%Y-%m-%d")
  S2TemporalFiltered = S2.filterDate(start_time, end_time);
  S2SpatialFiltered = S2TemporalFiltered.filterBounds(box);
  imagenes = S2SpatialFiltered.size()


  get_index = lambda image : ee.Feature(None,ee.Dictionary({
    "system_index":image.get('system:index')}))
  fc = S2SpatialFiltered.map(get_index)
  timestamps = []
  for f in fc.getInfo()['features']:
    attr = f['properties']['system_index']
    if attr is not None:
        timestamps.append(timestamp_from_system_index(attr))
  return timestamps


In [0]:
def process_file(input_file, output_file):
  df_points = load_n_process_measurements_file(input_file )
  df_output = pd.DataFrame()
  if not df_points.empty:
    for index, measure_row in df_points.iterrows():
        try:
          sat_row = get_satellite_dataframe_row(latitude=measure_row['LATITUDE'], longitude=measure_row['LONGITUDE'],date=measure_row['DATE'])
        except:
          print("get_satellite_dataframe_row failed, trying again")
          sat_row = get_satellite_dataframe_row(latitude=measure_row['LATITUDE'], longitude=measure_row['LONGITUDE'],date=measure_row['DATE'])
        if not sat_row.empty:
          merged_row = dict()
          merged_row.update(measure_row)
          merged_row.update(sat_row)
          df_merged_row = pd.DataFrame.from_dict(merged_row)
          df_output = pd.concat([df_output, df_merged_row], axis=0)   
  if df_output.empty:
    print("Warning: No valid points found in '{}".format(input_file))
  else:
    df_output['TIME_DIFF'] = df_output.apply(lambda row: abs(int(timestamp_from_system_index(row['system_index']) - row['TIMESTAMP'])), axis=1)
    df_output.sort_values(by=['TIME_DIFF'],inplace=True)
    df_output.drop_duplicates('system_index', keep='first',inplace=True)

  print("*** Complete. Saving file... '{}' ".format(output_file))
  df_output.to_csv(os.path.join(output_dir,output_file), index=False)

In [27]:
measurements_dir = "/content/drive/My Drive/Colab Notebooks/processed_2017-2019/blacksea_2017-2019"
output_dir = "/content/drive/My Drive/Colab Notebooks/sat_with_processed_S2-L2A/blacksea_2017-2019"

j = 0
for measurements_file_csv in os.listdir(measurements_dir):
    j = j+1
    print(j)
    output_file = "sat_with_" + measurements_file_csv
    if not os.path.isfile(os.path.join(output_dir,output_file)):
        print("~~ Processing file '{}' ~~".format(output_file))
        input_file = os.path.join(measurements_dir, measurements_file_csv)
        #process_file(input_file, output_file)
        
        try:
            process_file(input_file, output_file)
      
        except Exception as e:
            try:
                print("! Processing file failed, trying again")
                print(e)
                process_file(input_file, output_file)
            except Exception as e1:
                print("! Processing file failed again, giving up...")
                print(e1)
    else:
        print("~~ Skipping, file '{}' already exists.".format(output_file))

1
~~ Processing file 'sat_with_processed_BS_PR_CT_15AK_2017.csv' ~~
Opening measurement csv file '/content/drive/My Drive/Colab Notebooks/processed_2017-2019/blacksea_2017-2019/processed_BS_PR_CT_15AK_2017.csv'


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


*** Complete. Saving file... 'sat_with_processed_BS_PR_CT_15AK_2017.csv' 
2
~~ Processing file 'sat_with_processed_BS_TS_FB_Nc1158_2017.csv' ~~
Opening measurement csv file '/content/drive/My Drive/Colab Notebooks/processed_2017-2019/blacksea_2017-2019/processed_BS_TS_FB_Nc1158_2017.csv'
*** Complete. Saving file... 'sat_with_processed_BS_TS_FB_Nc1158_2017.csv' 
3
~~ Processing file 'sat_with_processed_BS_TS_FB_Nc1158_2018.csv' ~~
Opening measurement csv file '/content/drive/My Drive/Colab Notebooks/processed_2017-2019/blacksea_2017-2019/processed_BS_TS_FB_Nc1158_2018.csv'
*** Complete. Saving file... 'sat_with_processed_BS_TS_FB_Nc1158_2018.csv' 
4
~~ Skipping, file 'sat_with_processed_BS_TS_MO_EUXRo01.csv' already exists.
5
~~ Skipping, file 'sat_with_processed_BS_TS_MO_EUXRo02.csv' already exists.
6
~~ Skipping, file 'sat_with_processed_BS_TS_MO_EUXRo03.csv' already exists.
7
~~ Skipping, file 'sat_with_processed_BS_TS_MO_VarnaBuoySURF.csv' already exists.
8
~~ Skipping, file 'sat_w